In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler
import datetime

In [3]:
combined_data = pd.read_csv('../data/intermediates/pre_safegraph.csv')
combined_data.set_index('FIPS', inplace=True)
combined_data.head()

,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Jan Temp AVG / F,Feb Temp AVG / F,Mar Temp AVG / F,Apr Temp AVG / F,...,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,Area in square miles - Land area,NursingCt,HospCt,Hispanic Population,Black Population,Deaths,Outbreak_date
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,91.8,2.0,11.3,27.7,3.6,119.0,47.0,58.6,56.1,65.0,...,NaN,NaN,55601,594.44,6.0,85.0,1649.0,10915.0,4,2020-04-27
1003,114.6,3.0,9.7,31.3,3.6,115.5,52.0,62.0,59.7,66.0,...,11209.0,6503.0,218022,1589.78,9.0,152.0,10131.0,19492.0,6,2020-04-20
1011,17.5,6.0,24.8,13.3,4.7,58.7,48.0,58.2,56.6,64.3,...,NaN,NaN,10138,622.80,1.0,61.0,807.0,7168.0,8,2020-05-25
1013,27.0,6.0,15.4,16.1,4.8,74.9,48.4,59.3,57.0,64.4,...,NaN,NaN,19680,776.83,5.0,94.0,297.0,8839.0,16,2020-05-08
1015,195.7,3.0,15.9,18.0,4.7,91.0,44.2,52.6,53.0,63.0,...,4972.0,6890.0,114277,605.87,9.0,152.0,4469.0,24153.0,3,2020-04-20


In [4]:
# path = '/Users/beatrizmedeiros/Johns Hopkins/Fardin Ganjkhanloo - Number of trips/'

# for i in combined_data.index:
#     print(i)
#     outbreak_date = combined_data.loc[i, 'Outbreak_date']
#     trips = pd.read_csv(path + outbreak_date.strftime('%Y_%m_%d') + '.csv', index_col=0)
#     intra_mob = trips.loc[i, str(i)]
#     inter_mob = np.sum(trips.loc[i]) + np.sum(trips[str(i)])
#     print(intra_mob, inter_mob)
# #     print(trips.loc[i])
# #     print(trips[str(i)])
# #     print(trips.head())
#     break

In [5]:
path = '/Users/michaellepori/Documents/COVID/covid-drivers-master/data/intermediates/Number of trips/'
sg_mobility = pd.DataFrame(index=combined_data.index, columns=['2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility', '2wk Prior Inter-Mobility', '2wk Onset Inter-Mobility'])
# print(sg_mobility.head())
data_dict = {}
for i in combined_data.index: 
    print(i)
    row = combined_data.loc[i]
    outbreak_date = datetime.datetime.strptime(row['Outbreak_date'], '%Y-%m-%d')
    date1 = 14
    date2 = 7
    for j in range(-date1,date2): 
        mob_day = outbreak_date + datetime.timedelta(days=j) #.AddDays(j)
        mob_day = mob_day.date()
        if mob_day not in data_dict.keys():
            trips = pd.read_csv(path + mob_day.strftime('%Y_%m_%d') + '.csv', index_col=0)
            data_dict[mob_day] = trips
        
print(data_dict)

for i in combined_data.index: 
    print(i)
    inter_mob_per_day = np.zeros(21)
    intra_mob_per_day = np.zeros(21)
    row = combined_data.loc[i]
    outbreak_date = datetime.datetime.strptime(row['Outbreak_date'], '%Y-%m-%d')
    date1 = 14
    date2 = 7
    for j in range(-date1,date2): 
        mob_day = outbreak_date + datetime.timedelta(days=j) #.AddDays(j)
        mob_day = mob_day.date()
        try:
            trips = data_dict[mob_day]
        except: 
            print('missing mobility data for', mob_day.strftime('%Y_%m_%d'))
            if j > -date1: 
                inter_mob_per_day[j+date1] = inter_mob_per_day[j+date1-1]
                intra_mob_per_day[j+date1] = intra_mob_per_day[j+date1-1]
            else: 
                try: 
                    trips = pd.read_csv(path + outbreak_date.strftime('%Y_%m_%d') + '.csv', index_col=0)
                except: 
                    print('missing mobility data for outbreak date:', outbreak_date.strftime('%Y_%m_%d'))
                    continue
            continue
        intra_mob = trips.loc[i, str(i)]
        inter_mob = np.sum(trips.loc[i]) + np.sum(trips[str(i)]) - intra_mob # trips between this county and all OTHER counties 
        inter_mob_per_day[j+date1] = inter_mob
        intra_mob_per_day[j+date1] = intra_mob
    
    inter_prior_mob_arr = np.array(inter_mob_per_day[date1:date1+14])
    inter_onset_mob_arr = np.array(inter_mob_per_day[date1-7:-7])
    
    inter_prior_mob = np.mean(inter_prior_mob_arr)
    inter_onset_mob = np.mean(inter_onset_mob_arr)
    
    intra_prior_mob_arr = np.array(intra_mob_per_day[date1:date1+14])
    intra_onset_mob_arr = np.array(intra_mob_per_day[date1-7:-7])
    
    intra_prior_mob = np.mean(intra_prior_mob_arr)
    intra_onset_mob = np.mean(intra_onset_mob_arr)
    
    sg_mobility.loc[i,'2wk Prior Inter-Mobility'] = inter_prior_mob
    sg_mobility.loc[i, '2wk Onset Inter-Mobility'] = inter_onset_mob
    
    sg_mobility.loc[i,'2wk Prior Intra-Mobility'] = intra_prior_mob
    sg_mobility.loc[i, '2wk Onset Intra-Mobility'] = intra_onset_mob
#     if i < combined_data.index[5]:
#         print(mob_per_day)
#         print(prior_mob_arr, prior_mob)
#         print(onset_mob_arr, onset_mob)
#         print(pos_mob_arr, pos_mob)
sg_mobility.head()

1001
1003
1011
1013
1015
1017
1023
1041
1047
1049
1051
1053
1055
1059
1063
1065
1069
1073
1081
1085
1089
1091
1093
1095
1097
1101
1107
1111
1119
1123
1125
1129
1131
2020
4001
4005
4013
4015
4017
4019
4021
4025
4027
5023
5035
5045
5069
5075
5079
5119
5139
5143
6001
6013
6019
6025
6029
6037
6041
6047
6053
6055
6059
6061
6065
6067
6071
6073
6075
6077
6081
6083
6085
6089
6095
6097
6099
6107
6111
6113
8001
8005
8013
8014
8015
8031
8035
8037
8041
8051
8059
8063
8069
8075
8083
8085
8087
8101
8107
8123
9001
9003
9005
9007
9009
9011
9013
9015
10001
10003
10005
11001
12001
12003
12005
12009
12011
12013
12015
12017
12019
12021
12027
12031
12033
12035
12051
12053
12055
12057
12061
12065
12069
12071
12073
12079
12081
12083
12085
12086
12087
12091
12095
12097
12099
12101
12103
12105
12107
12109
12111
12113
12115
12117
12119
12121
12127
12131
13001
13009
13013
13015
13027
13029
13033
13035
13037
13045
13057
13059
13063
13067
13069
13071
13073
13077
13081
13087
13089
13095
13097
13099
13113
13115
1311

1003
1011
1013
1015
1017
1023
1041
1047
1049
1051
1053
1055
1059
1063
1065
1069
1073
1081
1085
1089
1091
1093
1095
1097
1101
1107
1111
1119
1123
1125
1129
1131
2020
4001
4005
4013
4015
4017
4019
4021
4025
4027
5023
5035
5045
5069
5075
5079
5119
5139
5143
6001
6013
6019
6025
6029
6037
6041
6047
6053
6055
6059
6061
6065
6067
6071
6073
6075
6077
6081
6083
6085
6089
6095
6097
6099
6107
6111
6113
8001
8005
8013
8014
8015
8031
8035
8037
8041
8051
8059
8063
8069
8075
8083
8085
8087
8101
8107
8123
9001
9003
9005
9007
9009
9011
9013
9015
10001
10003
10005
11001
12001
12003
12005
12009
12011
12013
12015
12017
12019
12021
12027
12031
12033
12035
12051
12053
12055
12057
12061
12065
12069
12071
12073
12079
12081
12083
12085
12086
12087
12091
12095
12097
12099
12101
12103
12105
12107
12109
12111
12113
12115
12117
12119
12121
12127
12131
13001
13009
13013
13015
13027
13029
13033
13035
13037
13045
13057
13059
13063
13067
13069
13071
13073
13077
13081
13087
13089
13095
13097
13099
13113
13115
13117
131

,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility
FIPS,,,,
1001,7461.57,7314.29,13927.1,13194.9
1003,35004,32185.7,47460.3,42444.7
1011,682.143,690.714,1401.29,1447.57
1013,2333.14,2367.29,4201.71,4058.29
1015,16956.9,16182.1,25290.6,23753.6


In [6]:
sg_mobility.to_csv('../data/processed/safegraph_mobility_avgs_ml.csv')